# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [19]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

import webpager as wp

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *




normalized_histos = list()

In [20]:
print (sys.path)

['..', 'python/', '..', 'python/', '/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools', '/opt/root/root-6.24.06_py310/lib', '/opt/local/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/opt/local/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/opt/local/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/cerminar/.virtualenvs/ntp-tools-py310/lib/python3.10/site-packages', '/opt/local/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages']


In [21]:
# %load samples.py
import ROOT
import pandas as pd



samples = [
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v152A.84C', 'ele-ideal'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v152A.84D', 'ele-emulated'),
]






In [22]:
wc = wp.WebPageCreator(top_dir='V84-EBTrackValidation', base_path='/Users/cerminar/CERNbox/www/plots/')

In [23]:
for smp in samples:
    smp.print_primitives()


<HistoFile ../plots/histos_doubleele_flat1to100_PU200_eg_v152A.84C.root, ele-ideal>
--- HistoClass: 
['EGHistos', 'EGResoHistos', 'HistoSetEff']
--- TPs: 
['TkEmEB', 'TkEleEB']
--- TP selections:
{   'TkEleEB': [   'IDTightEPt20',
                   'Pt40',
                   'IDTightEPt30',
                   'IDTightEPt10',
                   'Pt10',
                   'IDTightE',
                   'Pt20',
                   'IDTightEPt40',
                   'Pt30',
                   'all'],
    'TkEmEB': [   'IDTightPPt40',
                  'all',
                  'Pt10',
                  'Pt40',
                  'IDTightPPt20',
                  'Pt30',
                  'Pt20',
                  'IDTightPPt30',
                  'IDTightP',
                  'IDTightPPt10']}
--- GEN selections:
{   'TkEleEB': [   'GEN',
                   'GENPt15',
                   'GENEtaFABCD',
                   'GENEtaFABC',
                   'GENPt30',
                   'GENEtaF']

In [24]:
import python.collections as collections
import python.selections as selections
# this is needed to get all the labels defined in cfg modules
import python.plotters_config 
from cfg import *




labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



# #  ==== labels ===============================================
# tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
# tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
# gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]





In [25]:
# print (labels_dict['GENEeEtaA'])

# print (labels_dict.keys())

In [26]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.HistoSetEff)

    
# for tp in tps:
#     hplot.cache_histo(classtype=histos.HistoSetEff, 
#                       samples=samples,
#                       pus=[],
#                       tps=[tp],
#                       tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
#                       gen_sels=gen_select[tp])



--- EGHistos
# of plots: 960
# of primitives: 120
--- EGResoHistos
# of plots: 1200
# of primitives: 120
--- GenParticleHistos
# of plots: 1200
# of primitives: 120
--- EGHistos
# of plots: 960
# of primitives: 120
--- EGResoHistos
# of plots: 1200
# of primitives: 120
--- GenParticleHistos
# of plots: 1200
# of primitives: 120
CPU times: user 87.5 ms, sys: 4.44 ms, total: 91.9 ms
Wall time: 91.2 ms


In [27]:
for tp in hplot.tps():
    print ('TP: {}'.format(tp))
    print (hplot.data[hplot.data.tp == tp].tp_sel.unique())
    print (hplot.data[hplot.data.tp == tp].gen_sel.unique())

TP: TkEleEB
['Pt30' 'Pt10' 'IDTightEPt40' 'all' 'IDTightEPt10' 'Pt40' 'IDTightE'
 'IDTightEPt20' 'IDTightEPt30' 'Pt20']
['GENPt30' 'GENEtaF' 'GENEtaFABC' 'GEN' 'GENPt15' 'GENEtaFABCD']
TP: TkEmEB
['IDTightPPt30' 'IDTightPPt20' 'Pt30' 'IDTightPPt10' 'IDTightPPt40'
 'IDTightP' 'all' 'Pt10' 'Pt20' 'Pt40']
['GENEtaFABC' 'GENPt30' 'GENEtaFABCD' 'GENEtaF' 'GENPt15' 'GEN']


In [28]:
print (dir(samples[0]))
# samples[0].histo_file.ls()
samples

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'build_file_primitive_index', 'build_file_primitive_index_oldStyle', 'histo_file', 'histo_filename', 'label', 'name', 'oldStyle', 'open_file', 'print_file_primitive_index', 'print_primitives', 'type', 'version']


[<HistoFile ../plots/histos_doubleele_flat1to100_PU200_eg_v152A.84C.root, ele-ideal>,
 <HistoFile ../plots/histos_doubleele_flat1to100_PU200_eg_v152A.84D.root, ele-emulated>]

In [29]:
hplot.labels_dict['tkEmEB']

'TkEm EB'

In [30]:
hplot.labels_dict['TkEleELALL'] = 'TkEle'


In [31]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'TkEleELALL', ['Pt20'], ['GENEtaBC'], debug=False)
print (hsets)
hsets[0].name
hsets[0].name
dir(hsets[0].h_eff)

NameError: name 'sample' is not defined

In [ ]:
%%time

# ton_samples = hplot.data['sample'].unique()


for smp in hplot.data['sample'].unique():
    print ('Sample: {}'.format(smp))
    for pu in hplot.data[(hplot.data['sample'] == smp)].pu.unique():
        for tp in ['TkEmL2', 'TkEleL2']:
#         for tp in ['EG', 'TkEleEL']:
            print ('PU: {}, TP: {}'.format(pu, tp))
            for tp_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():
                if 'Pt' not in tp_sel:
                    continue
                if 'EGq2' in tp_sel or \
                   'EGq3' in tp_sel or \
                   'EGq4' in tp_sel:
                    continue
                    #                 tp_sel_den = 'all'
                # print tp_sel
                tp_sel_den = tp_sel.split('Pt')[0]
                if tp_sel_den == '':
                    tp_sel_den = 'all'
                print ('  {} den -> {}'.format(tp_sel, tp_sel_den)) 
                for gen_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp) & (hplot.data.tp_sel == tp_sel)].gen_sel.unique():
                    if gen_sel == 'nomatch' or 'Pt' in gen_sel:
                        continue
                    print(gen_sel)
                    hsetden = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel_den, gen_sel)
                    hset = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel, gen_sel)
                    hset[0][0].computeTurnOn(hsetden[0][0].h_num)


In [ ]:
hplot.data

In [32]:
hplot.data[hplot.data['sample'] == 'ele-V9'].iloc[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo


In [33]:
hplot.data['sample'].unique()

array(['ele-ideal', 'ele-emulated'], dtype=object)

In [34]:
hplot.data['tp'].unique()

array(['TkEleEB', 'TkEmEB'], dtype=object)

In [35]:
hplot.data[hplot.data.tp == 'L1TrkEmu'].tp_sel.unique()

array([], dtype=object)

In [36]:
hplot.data[hplot.data.tp == 'TkEleEL'].tp_sel.unique()

array([], dtype=object)

In [37]:
# sample='ele'

In [38]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5Pt25', ['GENEtaBC'], debug=False)
print (hsets)
hsets[0].name
print (dir(hsets[0].h_eff))


NameError: name 'sample' is not defined

In [39]:
newconfig=rleg_config
# newconfig=tdr_config

newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)

## Effinciency vs $\eta$

In [40]:
# newconfig=tdr_config
# newconfig.legend_size=(0.5, 0.2)
# newconfig.legend_position=(0.4, 0.2)

# dm = DrawMachine(newconfig)
# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele-V9'], ['PU200'], ['EG'], 'EGq5Pt25', 'GENPt30', debug=False)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['EG calo-only'])

# # hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid'], ['PU200'], ['L1Trk', ], ['Pt10'], 'GENPt30', debug=False)
# # dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['L1 tracks (p_{T} > 10 GeV)'])
# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele-V9'], ['PU200'], ['TkEle', 'TkEleEL'], 'EGq5Pt25', 'GENPt30', debug=False)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['EG track-match', 'EG track-match (Elliptic cut)'])


# dm.draw(text='EG endcap, p_{T}^{GEN}>25GeV', x_min=1.5, x_max=3.2, y_min=0.5, y_max=1.1, v_lines=[1.52, 2.4])
# dm.write(name='egEndcap_matchingEffBreakdown')

In [41]:
hplot.data[hplot.data.tp == 'TkEmEE']


,sample,pu,tp,tp_sel,gen_sel,classtype,histo


In [42]:
dm = DrawMachine(newconfig)
dm.config.legend_position = (0.5,0.15)
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ideal', 'ele-emulated'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt15', debug=False)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')

dm.toWeb(name='hEffVsEta_TkEmEB_all_GENPt15', page_creator=wc)
dm.canvas.SaveAs("test.png")


Info in <TCanvas::Print>: png file test.png has been created


In [43]:
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ideal', 'ele-emulated'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['all'], 'GENPt15', debug=False)
print(hsets)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, 
        v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')


dm.toWeb(name='hEffVsEta_TkEleEB_all_GENPt15', page_creator=wc)



[<python.l1THistos.HistoSetEff object at 0x2ad577880>, <python.l1THistos.HistoSetEff object at 0x2ae829780>]


In [44]:
wc.publish(directory='eff')

Yes/No: 
Yes/No: 
Yes/No: 
Yes/No: 
Yes/No: 
Yes/No: yes
publishing canvas: hEffVsEta_TkEmEB_all_GENPt15
publishing canvas: hEffVsEta_TkEleEB_all_GENPt15


Info in <TCanvas::Print>: png file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEmEB_all_GENPt15.png has been created
Info in <TCanvas::Print>: pdf file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEmEB_all_GENPt15.pdf has been created
Info in <TCanvas::Print>: png file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEleEB_all_GENPt15.png has been created
Info in <TCanvas::Print>: pdf file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEleEB_all_GENPt15.pdf has been created


In [45]:
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ideal', 'ele-emulated'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['all'], 'GENPt30', debug=False)
print(hsets)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')

dm.toWeb(name='hEffVsEta_TkEleEB_all_GENPt30', page_creator=wc)




[<python.l1THistos.HistoSetEff object at 0x2ad4f69b0>, <python.l1THistos.HistoSetEff object at 0x2ad4f4d60>]


In [46]:
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ideal', 'ele-emulated'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['IDTightE'], 'GENPt15', debug=False)
print(hsets)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')


dm.toWeb(name='hEffVsEta_TkEleEB_IDTight_GENPt15', page_creator=wc)



[<python.l1THistos.HistoSetEff object at 0x2ae828a60>, <python.l1THistos.HistoSetEff object at 0x2ae8c89d0>]


In [47]:
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ideal', 'ele-emulated'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['IDTightE'], 'GENPt30', debug=False)
print(hsets)

# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=1.5, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)

# dm.write(name='test')

dm.toWeb(name='hEffVsEta_TkEleEB_IDTight_GENPt30', page_creator=wc)




[<python.l1THistos.HistoSetEff object at 0x2ad5b61a0>, <python.l1THistos.HistoSetEff object at 0x2ad5b70d0>]


In [48]:
wc.publish(directory='eff')

Yes/No: 
Yes/No: 
Yes/No: 
Yes/No: yes
publishing canvas: hEffVsEta_TkEmEB_all_GENPt15
publishing canvas: hEffVsEta_TkEleEB_all_GENPt15
publishing canvas: hEffVsEta_TkEleEB_all_GENPt30
publishing canvas: hEffVsEta_TkEleEB_IDTight_GENPt15
publishing canvas: hEffVsEta_TkEleEB_IDTight_GENPt30


Info in <TCanvas::Print>: png file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEmEB_all_GENPt15.png has been created
Info in <TCanvas::Print>: pdf file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEmEB_all_GENPt15.pdf has been created
Info in <TCanvas::Print>: png file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEleEB_all_GENPt15.png has been created
Info in <TCanvas::Print>: pdf file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEleEB_all_GENPt15.pdf has been created
Info in <TCanvas::Print>: png file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEleEB_all_GENPt30.png has been created
Info in <TCanvas::Print>: pdf file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEleEB_all_GENPt30.pdf has been created
Info in <TCanvas::Print>: png file /Users/cerminar/CERNbox/www/plots/V84-EBTrackValidation/eff/hEffVsEta_TkEleEB_IDTight_GENPt

In [20]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq1'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=False,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





No match found for: sample: ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'] pu: ['PU200'], tp: ['EGStaEE'], tp_sel: ['EGq1'], gen_sel: GENEePt30, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [14]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq3'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=False,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





No match found for: sample: ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'] pu: ['PU200'], tp: ['EGStaEE'], tp_sel: ['EGq3'], gen_sel: GENEePt30, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [15]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho-12.3', 'pho-21.3.tp', 'pho-11.1', 'pho-11.1.pf'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq3'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)




dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.8,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





No match found for: sample: ['pho-12.3', 'pho-21.3.tp', 'pho-11.1', 'pho-11.1.pf'] pu: ['PU200'], tp: ['EGStaEE'], tp_sel: ['EGq3'], gen_sel: GENEePt30, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [14]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['EGq3', 'IDTightP'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 3.2
0.0 3.2
0.0 3.2


In [15]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['IDTightP'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEmL2'], 
    ['all'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





***Warning: Ratio plot can not be added: [RatioPlot] num and den TGraphAsymmErrors objs have different # of points: (29 and 31 resp.)


In [16]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho', 'pho-ref'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['EGq3', 'IDTightP'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho', 'pho-ref'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 3.2
0.0 3.2
0.0 3.2


In [17]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['IDTightP'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho'], 
    ['PU200'], 
    ['TkEmL2'], 
    ['all'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 3.2
0.0 3.2
0.0 3.2


In [18]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['EGq3', 'IDTightP'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 3.2
0.0 3.2
0.0 3.2


In [19]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['IDTightP'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEmL2'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





***Warning: Ratio plot can not be added: [RatioPlot] num and den TGraphAsymmErrors objs have different # of points: (29 and 31 resp.)


In [20]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho', 'pho-ref'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['EGq3', 'IDTightP'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho', 'pho-ref'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 3.2
0.0 3.2
0.0 3.2


In [21]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['IDTightP'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEmL2'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 3.2
0.0 3.2
0.0 3.2


In [22]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3', 'IDTightE'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)
# print(text.split('EB'))
text = '{} {}'.format(text.split('EB')[0], text.split('EB')[1])

dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.65, y_max=1.1, 
        v_lines=[1.52, 1.7, 2.4],
        h_lines=[1.],
        do_ratio=False,
        y_min_ratio=0.9,
        y_max_ratio=1.1,
        y_axis_label='L1 eff.')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [23]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['IDTightE'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['all'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEleL2'], 
    ['all', 'IDTightE'], 'GENPt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





***Warning: Ratio plot can not be added: [RatioPlot] num and den TGraphAsymmErrors objs have different # of points: (29 and 31 resp.)


In [39]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3', 'IDTightE'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['LooseTkID', 'IDTightE'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=False,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [25]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEleL2'], 
    ['all', 'IDTightE'], 'GENPt30', debug=False)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=False,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [40]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3'], 'GENEePt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['all'], 'GENPt30', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=labels)


dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text=text, 
        x_min=0., x_max=3.2, 
        y_min=0.5, y_max=1.1, v_lines=[1.52, 1.7, 2.4],
       do_ratio=False,
       y_min_ratio=0.9,
       y_max_ratio=1.1)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





No match found for: sample: ['ele-11.1', 'ele-11.1.pf'] pu: ['PU200'], tp: ['TkEleEE'], tp_sel: ['EGq3'], gen_sel: GENEePt30, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [47]:
def compute_average(name, graph, x_min, x_max):
    npoints = graph.GetN()
    x_buf = graph.GetX()
    x_buf.reshape((npoints,))
    x_data = array.array('d', x_buf)
    y_buf = graph.GetY()
    y_buf.reshape((npoints,))
    y_data = array.array('d', y_buf)
    eff_sum = 0.
    eff_n = 0

    for binn in range(1, npoints):
        if x_data[binn] >= x_min and x_data[binn]<= x_max:
            eff_sum += y_data[binn]
            eff_n += 1
    avg_eff = eff_sum/eff_n
    print (f'graph: {name} avg eff.: {avg_eff}')


## Efficiency vs $p_{T}$

In [49]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEmEB'], 
    ['all'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [56]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq1', 'IDTightS'], 
    ['GENEeEtaBC'], debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [41]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq1'], 
    'GENEeEtaA', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [42]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq1'], 
    'GENEeEtaB', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [43]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq1'], 
    'GENEeEtaC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [44]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref'], 
    ['PU200'], 
    ['EGStaEE'], 
    ['EGq1'], 
    'GENEeEtaD', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [46]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3', 'IDTightE', 'all'], 
    'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





***Warning: Ratio plot can not be added: list index out of range
0.0 100.0
0.0 100.0
0.0 100.0


In [47]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3'], 
    'GENEeEtaB', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [48]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3'], 
    'GENEeEtaC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [49]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3'], 
    'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [50]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele', 'ele-ref'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['all'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [52]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-ref', 'ele'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['EGq3', 'IDTightP', 'all'], 
    'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 100.0
0.0 100.0
0.0 100.0


In [54]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho-ref', 'pho'], 
    ['PU200'], 
    ['TkEmEE'], 
    ['EGq3', 'IDTightP', 'all'], 
    'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





***Warning: Ratio plot can not be added: list index out of range
0.0 100.0
0.0 100.0
0.0 100.0


In [ ]:
TP: TkEleEB
['Pt20' 'LooseTkID@20kHz' 'LooseTkID' 'Pt10' 'all' 'LooseTkID@100kHz'
 'LooseTkIDPt20' 'EtaFLooseTkID@100kHz' 'LooseTkID@1000kHz'
 'EtaFLooseTkID@1000kHz' 'EtaFLooseTkID@20kHz' 'LooseTkIDPt10' 'IDTightE'
 'IDTightEPt20' 'IDTightEPt10']
['GENEtaF' 'GENPt30' 'GEN' 'GENPt15' 'GENEtaFABC' 'GENEtaFABCD']
TP: TkEmEB
['Pt20' '@1000kHz' 'EtaF@20kHz' 'Pt10' 'all' 'EtaF@100kHz' 'EtaF@1000kHz'
 '@100kHz' 'LooseTkIDPt20' 'LooseTkID' 'LooseTkIDPt10' '@20kHz'
 'IDTightPPt10' 'IDTightP' 'IDTightPPt20']
['GENEtaF' 'GENPt30' 'GEN' 'GENPt15' 'GENEtaFABC' 'GENEtaFABCD']
TP: TkEmEE
['EGq3EtaBC@100kHz' 'EGq3Pt10' 'EGq3Pt20' 'EGq3' 'EGq1Pt20'
 'EGq3EtaBC@1000kHz' 'EGq1' 'EGq1Pt10' 'EGq3EtaBC@20kHz' 'Pt10' 'Pt20'
 'IDTightPPt20' 'all' 'IDTightPPt10' 'IDTightP']
['GENEePt15' 'GENEeEtaBC' 'GENEeEtaC' 'GENEe' 'GENEeEtaB' 'GENEeEtaA'
 'GENEePt30']
TP: EGStaEE
['EGq1' 'EGq1Pt10' 'EGq1Pt20' 'EGq3Pt10' 'EGq3' 'EGq3Pt20']
['GENEeEtaC' 'GENEeEtaB' 'GENEe' 'GENEeEtaBC' 'GENEeEtaD' 'GENEeEtaA'
 'GENEeEtaBCD' 'GENEePt15' 'GENEePt30']
TP: TkEleEE
['EGq1' 'EGq3EtaBC@100kHz' 'EGq1Pt10' 'EGq3Pt20' 'EGq3EtaBC@1000kHz'
 'EGq1Pt20' 'EGq3Pt10' 'EGq3' 'EGq3EtaBC@20kHz' 'all' 'IDTightEPt10'
 'IDTightE' 'IDTightEPt20' 'Pt10' 'Pt20']
['GENEeEtaC' 'GENEeEtaB' 'GENEeEtaBC' 'GENEePt30' 'GENEePt15' 'GENEe'
 'GENEeEtaA']
TP: TkEleL2
['EtaABC@1000kHz' 'IDTightEEtaABC@20kHz' 'IDTightEEtaFABC@1000kHz'
 'EtaABC@20kHz' 'IDTightE' 'EtaABC@100kHz' 'EtaFABC@100kHz' 'EtaF@100kHz'
 'IDTightEEtaF@100kHz' 'IDTightEEtaF@1000kHz' 'IDTightEEtaABC@1000kHz'
 'IDTightEPt20' 'EtaFABC@1000kHz' 'Pt20' 'IDTightEEtaFABC@100kHz' 'Pt10'
 'EtaF@20kHz' 'EtaFABC@20kHz' 'IDTightEEtaFABC@20kHz'
 'IDTightEEtaABC@100kHz' 'IDTightEPt10' 'IDTightEEtaF@20kHz'
 'EtaF@1000kHz' 'all']
['GENEePt30' 'GEN' 'GENPt30' 'GENEePt15' 'GENEtaBCD' 'GENPt15' 'GENEtaD'
 'GENEtaF' 'GENEtaABC' 'GENEtaFABC' 'GENEtaBC']
TP: TkEmL2
['IDTightPEtaF@20kHz' 'all' 'EtaF@100kHz' 'IDTightPEtaABC@100kHz'
 'EtaF@1000kHz' 'IDTightPEtaABC@1000kHz' 'IDTightPEtaFABC@100kHz'
 'EtaFABC@100kHz' 'EtaF@20kHz' 'IDTightPEtaF@1000kHz'
 'IDTightPEtaFABC@1000kHz' 'Pt10' 'EtaFABC@20kHz' 'IDTightPPt10'
 'IDTightPEtaABC@20kHz' 'IDTightPEtaF@100kHz' 'EtaABC@20kHz'
 'EtaABC@1000kHz' 'EtaABC@100kHz' 'IDTightPEtaFABC@20kHz' 'IDTightP'
 'EtaFABC@1000kHz' 'IDTightPPt20' 'Pt20']
['GENPt30' 'GENEtaD' 'GENEePt30' 'GEN' 'GENEtaF' 'GENEePt15' 'GENPt15'
 'GENEtaBCD' 'GENEtaBC' 'GENEtaFABC' 'GENEtaABC']

In [95]:
import python.calibrations as calib
calib_mgr = calib.CalibManager()
calib_mgr.set_pt_wps_version('data/rate_pt_wps_v152B.82B.json')
rate_pt_wps = calib_mgr.get_pt_wps()
data_selections = calib.rate_pt_wps_selections(
rate_pt_wps, 'TkEmL2')
# data_selections = calib.rate_pt_wps_selections(
# rate_pt_wps, 'TkEleL2')

# print(data_selections)
selm = selections.SelectionManager()
# labels_dict.update(selm.get_labels())
hplot.labels_dict.update(selm.get_labels())


[CalibManager] pt wps json file version: data/rate_pt_wps_v152B.82B.json!
[CalibManager] reading pt wps json file data/rate_pt_wps_v152B.82B.json!


In [96]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho'], 
    ['PU200'], 
    ['TkEmL2'], 
    ['IDTightPEtaABC@20kHz', 'EtaABC@20kHz'], 
    'GENEtaABC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 100.0
0.0 100.0
0.0 100.0


In [97]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['pho'], 
    ['PU200'], 
    ['TkEmL2'], 
    ['IDTightPEtaABC@100kHz', 'EtaABC@100kHz'], 
    'GENEtaABC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 100.0
0.0 100.0
0.0 100.0


In [99]:
import python.calibrations as calib
calib_mgr = calib.CalibManager()
calib_mgr.set_pt_wps_version('data/rate_pt_wps_v152B.82B.json')
rate_pt_wps = calib_mgr.get_pt_wps()
data_selections = calib.rate_pt_wps_selections(
rate_pt_wps, 'TkEleL2')
# data_selections = calib.rate_pt_wps_selections(
# rate_pt_wps, 'TkEleL2')

# print(data_selections)
selm = selections.SelectionManager()
# labels_dict.update(selm.get_labels())
hplot.labels_dict.update(selm.get_labels())


[CalibManager] pt wps json file version: data/rate_pt_wps_v152B.82B.json!
[CalibManager] reading pt wps json file data/rate_pt_wps_v152B.82B.json!


In [100]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEleL2'], 
    ['IDTightEEtaABC@20kHz', 'EtaABC@20kHz'], 
    'GENEtaABC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=False,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [60]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele'], 
    ['PU200'], 
    ['TkEleL2'], 
    ['IDTightEEtaABC@100kHz', 'EtaABC@100kHz'], 
    'GENEtaABC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 100.0
0.0 100.0
0.0 100.0


In [82]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['photon-ref', 'photon-fw'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq1', 'EGq2or3'], 
    'GENEeEtaB', debug=False)
for hset in hsets:
    hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 100.0
0.0 100.0
0.0 100.0


In [80]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['photon-ref', 'photon-fw'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq1', 'EGq2or3'], 
    'GENEeEtaC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text, 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=False,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [41]:
for tp in tps:
    print ('--- TP: {}'.format(tp))
    primitives = histo_primitives[histo_primitives.tp == tp].tp_sel.unique()
    print (histo_primitives[histo_primitives.tp == tp].gen_sel.unique())
    for rate in [20, 100, 1000]:
        print(f'-- rate: {rate}kHz')
        rate_sels = [sel for sel in primitives if f'{rate}kHz' in sel]
        for wp in [95, 98]:
            wp_sel = [sel for sel in rate_sels if f'WP{wp}' in sel]
            print(f'- WP: {wp}%')
            print (wp_sel)
        print([sel for sel in rate_sels if 'WP' not in sel])   
#     print (histo_primitives[histo_primitives.tp == tp].tp_sel.unique())


NameError: name 'tps' is not defined

In [45]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3EtaBC@20kHz'], 
    'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text + " @ 20kHz", 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
        do_ratio=True,
        y_min_ratio=0.9,
        y_max_ratio=1.2,
        y_axis_label='eff.')
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [112]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['TkEleEE'], 
    ['EGq3EtaBC@100kHz'], 
    'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text + " @ 100kHz", 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [111]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['LooseTkID@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text + " @ 20kHz", 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [114]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['LooseTkID@100kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text + " @ 100kHz", 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [136]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.1)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['ele-12.3', 'ele-12.3.tp', 'ele-11.1', 'ele-11.1.pf'], 
    ['PU200'], 
    ['TkEleEB'], 
    ['LooseTkID@1000kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)
# dm.addRatioHisto(2,0)
# dm.addRatioHisto(3,0)
# dm.addRatioHisto(4,0)


dm.draw(text=text + " @ 1000kHz", 
        x_min=0, x_max=100, 
        y_min=0.0, y_max=1.1, 
        h_lines=[1.0, 0.9],
       do_ratio=True,
       y_min_ratio=0.9,
       y_max_ratio=1.2)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





### Barrel

#### ISO+p_{T} WP @ 1000kHz

In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@1000kHz', 
     'tkIsoWP98Pt10@1000kHz', 
     'tkIsoWP98Pt20@1000kHz', 
     'tkIsoWP98Pt40@1000kHz', 
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text)


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@1000kHz', 
     'tkIsoWP95Pt10@1000kHz', 
     'tkIsoWP95Pt20@1000kHz', 
     'tkIsoWP95Pt40@1000kHz', 
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text)


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@1000kHz', 
     'tkIsoPVWP98Pt10@1000kHz', 
#      'tkIsoPVWP95Pt10@1000kHz', 
     'tkIsoPVWP98Pt20@1000kHz', 
     'tkIsoPVWP98Pt40@1000kHz', 

     #      'tkIsoPVWP95Pt20@1000kHz'
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)




dm.draw(text=text)


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dph-V11'], ['PU200'], ['PFNFtkEmEB'], [
    '@1000kHz', 
    'pfIsoWP98Pt10@1000kHz', 
#     'pfIsoWP95Pt10@1000kHz', 
    'pfIsoWP98Pt20@1000kHz', 
    'pfIsoWP98Pt40@1000kHz', 
    #     'pfIsoWP95Pt20@1000kHz'
], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text)


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dph-V11'], ['PU200'], ['PFNFtkEmEB'], [
    '@1000kHz', 
    'pfIsoPVWP98Pt10@1000kHz', 
#     'pfIsoPVWP95Pt10@1000kHz', 
    'pfIsoPVWP98Pt20@1000kHz',
    'pfIsoPVWP98Pt40@1000kHz', 

#     'pfIsoPVWP95Pt20@1000kHz'
], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@1000kHz', 
     'tkIsoWP98Pt10@1000kHz', 
     'tkIsoPVWP98Pt10@1000kHz', 
     'pfIsoWP98Pt10@1000kHz', 
     'pfIsoPVWP98Pt10@1000kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dph-V11'], ['PU200'], ['PFNFtkEmEB'], ['@1000kHz', 'tkIsoWP98Pt10@1000kHz', 'tkIsoPVWP98Pt10@1000kHz', 'pfIsoWP98Pt10@1000kHz', 'pfIsoPVWP98Pt10@1000kHz'], 'GENEtaF', debug=False)
for hset in hsets:
    hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, y_min=0.8, y_max=1.1, h_lines=[1., 0.95])


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@1000kHz', 'tkIsoWP98Pt10@1000kHz', 'tkIsoWP95Pt10@1000kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text)


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@1000kHz', 'tkIsoPVWP98Pt10@1000kHz', 'tkIsoPVWP95Pt10@1000kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text)


#### ISO+p_{T} WP @ 20kHz 

In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'tkIsoWP98Pt10@20kHz', 
     'tkIsoWP98Pt20@20kHz', 
     'tkIsoWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'tkIsoPVWP98Pt10@20kHz', 
     'tkIsoPVWP98Pt20@20kHz', 
     'tkIsoPVWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'pfIsoWP98Pt10@20kHz', 
     'pfIsoWP98Pt20@20kHz', 
     'pfIsoWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'pfIsoPVWP98Pt10@20kHz', 
     'pfIsoPVWP98Pt20@20kHz', 
     'pfIsoPVWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'tkIsoWP95Pt40@20kHz', 
     'tkIsoWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'tkIsoPVWP95Pt40@20kHz', 
     'tkIsoPVWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'pfIsoWP95Pt40@20kHz', 
     'pfIsoWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'tkIsoWP98Pt40@20kHz', 
     'tkIsoPVWP98Pt40@20kHz', 
     'pfIsoWP98Pt40@20kHz', 
     'pfIsoPVWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@20kHz', 
     'tkIsoWP98Pt40@20kHz', 
     'tkIsoPVWP98Pt40@20kHz', 
     'pfIsoWP98Pt40@20kHz', 
     'pfIsoPVWP98Pt40@20kHz'], 
    'GENEtaF', debug=False)

for hset in hsets:
    hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, y_min=0.8, y_max=1.1, h_lines=[1., 0.95])


#### ISO+p_{T} WP @ 100kHz

In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'tkIsoWP98Pt10@100kHz', 
     'tkIsoWP98Pt20@100kHz', 
     'tkIsoWP98Pt40@100kHz', 
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'tkIsoPVWP98Pt10@100kHz', 
     'tkIsoPVWP98Pt20@100kHz', 
     'tkIsoPVWP98Pt40@100kHz', 
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'pfIsoWP98Pt10@100kHz', 
     'pfIsoWP98Pt20@100kHz', 
     'pfIsoWP98Pt40@100kHz', 
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'pfIsoPVWP98Pt10@100kHz', 
     'pfIsoPVWP98Pt20@100kHz', 
     'pfIsoPVWP98Pt40@100kHz', 
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'tkIsoWP95Pt20@100kHz',
     'tkIsoWP98Pt20@100kHz',
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'tkIsoPVWP95Pt20@100kHz',
     'tkIsoPVWP98Pt20@100kHz',
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'pfIsoWP95Pt20@100kHz',
     'pfIsoWP98Pt20@100kHz',
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'pfIsoPVWP95Pt20@100kHz',
     'pfIsoPVWP98Pt20@100kHz',
    ], 'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'tkIsoWP95Pt20@100kHz', 
     'tkIsoPVWP95Pt20@100kHz', 
     'pfIsoWP95Pt20@100kHz', 
     'pfIsoPVWP95Pt20@100kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 
     'tkIsoWP98Pt20@100kHz', 
     'tkIsoPVWP98Pt20@100kHz', 
     'pfIsoWP98Pt20@100kHz', 
     'pfIsoPVWP98Pt20@100kHz'], 
    'GENEtaF', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEB'], 
    ['@100kHz', 'tkIsoWP98Pt20@100kHz', 'tkIsoPVWP98Pt20@100kHz', 'pfIsoWP98Pt20@100kHz', 'pfIsoPVWP98Pt20@100kHz'], 
    'GENEtaF', debug=False)
for hset in hsets:
    hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95], y_min=0.8, y_max=1.1)


### Endcap

In [ ]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.4, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['tkEmEE'], 
    ['EGq4'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['sim'])

print (text)
hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq6'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['emu EG-ID'])


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq2or3'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['emu EG-ID noPU'])


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq1or3'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['emu PF EM-ID noPU'])


dm.addRatioHisto(0,1)
dm.addRatioHisto(2,1)
dm.addRatioHisto(3,1)

dm.draw(text='EG, Tk-em, 1.52 < |#eta^{GEN}| <= 2.4', 
        y_min=0.5, y_max=1.1, h_lines=[.95, 1.],
       do_ratio=True)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [ ]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.4, 0.15)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['tkEleEE'], 
    ['EGq4'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['sim'])


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['PFNFtkEleEE'], 
    ['EGq6'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['emu EG-ID'])


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['PFNFtkEleEE'], 
    ['EGq2or3'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['emu EG-ID noPU'])


hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dele-V11'], 
    ['PU200'], 
    ['PFNFtkEleEE'], 
    ['EGq1or3'], 'GENEeEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels=['emu PF EM-ID noPU'])

dm.addRatioHisto(0,1)
dm.addRatioHisto(2,1)
dm.addRatioHisto(3,1)

dm.draw(text='EG, Tk-electron, 1.52 < |#eta^{GEN}| <= 2.4', 
        y_min=0., y_max=1., h_lines=[.95, 1.],
       do_ratio=True)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [ ]:
for tp in ['PFNFtkEmEE']:
    print ('--- TP: {}'.format(tp))
    primitives = histo_primitives[histo_primitives.tp == tp].tp_sel.unique()
    print (histo_primitives[histo_primitives.tp == tp].gen_sel.unique())
    for rate in [20, 100, 1000]:
        print(f'-- rate: {rate}kHz')
        rate_sels = [sel for sel in primitives if f'{rate}kHz' in sel]
        for wp in [95, 98]:
            wp_sel = [sel for sel in rate_sels if f'WP{wp}' in sel]
            print(f'- WP: {wp}%')
            print (wp_sel)
        print([sel for sel in rate_sels if 'WP' not in sel])   
#     print (histo_primitives[histo_primitives.tp == tp].tp_sel.unique())


#### ISO+p_{T} WP @ 100kHz

In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5tkIsoWP98Pt10EtaABC@100kHz', 
     'EGq5tkIsoWP98Pt20EtaABC@100kHz', 
     'EGq5tkIsoWP98Pt40EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5tkIsoPVWP98Pt10EtaABC@100kHz', 
     'EGq5tkIsoPVWP98Pt20EtaABC@100kHz', 
     'EGq5tkIsoPVWP98Pt40EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5pfIsoWP98Pt10EtaABC@100kHz', 
     'EGq5pfIsoWP98Pt20EtaABC@100kHz', 
     'EGq5pfIsoWP98Pt40EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5pfIsoPVWP98Pt10EtaABC@100kHz', 
     'EGq5pfIsoPVWP98Pt20EtaABC@100kHz', 
     'EGq5pfIsoPVWP98Pt40EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5tkIsoWP95Pt10EtaABC@100kHz', 
     'EGq5tkIsoWP98Pt10EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5tkIsoPVWP95Pt10EtaABC@100kHz', 
     'EGq5tkIsoPVWP98Pt10EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5pfIsoWP95Pt10EtaABC@100kHz', 
     'EGq5pfIsoWP98Pt10EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5pfIsoPVWP95Pt10EtaABC@100kHz', 
     'EGq5pfIsoPVWP98Pt10EtaABC@100kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@100kHz', 
     'EGq5tkIsoWP98Pt10EtaABC@100kHz', 
     'EGq5tkIsoPVWP98Pt10EtaABC@100kHz', 
     'EGq5pfIsoWP98Pt10EtaABC@100kHz', 
     'EGq5pfIsoPVWP98Pt10EtaABC@100kHz', 
    ], 
    'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


#### ISO+p_{T} WP @ 20kHz

In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5tkIsoWP98Pt10EtaABC@20kHz', 
     'EGq5tkIsoWP98Pt20EtaABC@20kHz', 
     'EGq5tkIsoWP98Pt40EtaABC@20kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5tkIsoPVWP98Pt10EtaABC@20kHz', 
     'EGq5tkIsoPVWP98Pt20EtaABC@20kHz', 
     'EGq5tkIsoPVWP98Pt40EtaABC@20kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5pfIsoWP98Pt10EtaABC@20kHz', 
     'EGq5pfIsoWP98Pt20EtaABC@20kHz', 
     'EGq5pfIsoWP98Pt40EtaABC@20kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5pfIsoPVWP98Pt10EtaABC@20kHz', 
     'EGq5pfIsoPVWP98Pt20EtaABC@20kHz', 
     'EGq5pfIsoPVWP98Pt40EtaABC@20kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5pfIsoWP95Pt20EtaABC@20kHz', 
     'EGq5pfIsoWP98Pt20EtaABC@20kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5pfIsoPVWP95Pt10EtaABC@20kHz', 
     'EGq5pfIsoPVWP98Pt10EtaABC@20kHz', 
    ], 'GENEtaBC', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5tkIsoWP98Pt10EtaABC@20kHz', 
     'EGq5tkIsoPVWP98Pt10EtaABC@20kHz', 
     'EGq5pfIsoWP98Pt10EtaABC@20kHz', 
     'EGq5pfIsoPVWP98Pt10EtaABC@20kHz', 
    ], 
    'GENEtaBC', debug=False)
for hset in hsets:
    hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5tkIsoWP98Pt20EtaABC@20kHz', 
     'EGq5tkIsoPVWP98Pt20EtaABC@20kHz', 
     'EGq5pfIsoWP98Pt20EtaABC@20kHz', 
     'EGq5pfIsoPVWP98Pt20EtaABC@20kHz', 
    ], 
    'GENEtaBC', debug=False)
for hset in hsets:
    hset.computeEff(rebin=2)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, h_lines=[1., 0.95])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(
    histos.HistoSetEff, 
    ['dph-V11'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq5EtaABC@20kHz', 
     'EGq5tkIsoWP98Pt20EtaABC@20kHz', 
     'EGq5tkIsoPVWP98Pt20EtaABC@20kHz', 
     'EGq5pfIsoWP98Pt20EtaABC@20kHz', 
     'EGq5pfIsoPVWP98Pt20EtaABC@20kHz', 
    ], 
    'GENEtaBC', debug=False)
for hset in hsets:
    hset.computeEff(rebin=4)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)

dm.draw(text=text, y_min=0.85, y_max=1.05, h_lines=[1., 0.95])


In [ ]:
line_y1p0 = ROOT.TLine(0.,1., 110., 1.)
line_y0p95 = ROOT.TLine(0.,0.95, 110., 0.95)
line_y0p95.SetLineStyle(2)

In [ ]:
hplot.labels_dict['TkEleEL'] = hplot.labels_dict['tkEleEE']
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11'], ['PU200'], ['PFEgEE','PFNFEgEE','EgEE'], 'EGq5', 'GENEtaBCD', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11'], ['PU200'], ['tkEleEE','PFtkEleEE','PFNFtkEleEE'], 'EGq5', 'GENEtaBC', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], ['TDR sim.', 'emu. v0', 'emu. v1'], text=text+', TkElectron EE', min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11'], ['PU200'], ['tkEmEE', 'PFtkEmEE', 'PFNFtkEmEE' ,], 'EGq5', 'GENEtaBC', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], ['TDR sim.', 'emu. v0', 'emu v1'], text=text+', TkEm EE', min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
# hplot.labels_dict['TkEleEL'] = hplot.labels_dict['tkEleEE']
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11'], ['PU200'], ['tkEleEB','PFtkEleEB','PFNFtkEleEB'], 'LooseTkID', 'GENEtaF', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], ['TDR sim.', 'emu. v0', 'emu. v1'], text=text+', TkElectron EB', min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
# hplot.labels_dict['TkEleEL'] = hplot.labels_dict['tkEleEE']
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11'], ['PU200'], ['tkEmEB', 'PFtkEmEB','PFNFtkEmEB'], 'all', 'GENEtaF', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], ['TDR sim', 'emu. v0', 'emu. v1'], text=text+' TkEm EB', min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
hplot.labels_dict['TkEleEL'] = hplot.labels_dict['tkEleEE']
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11', 'dele-NM','ele-V9'], ['PU200'], ['TkEleEL','tkEleEE'], 'EGq5', 'GENEtaBC', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleEL', 'EGq5Pt25', 'GENEtaBC', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11', 'ele-V9'], ['PU200'], 'TkEleELALL', 'LooseTkID', 'GENEtaF', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleELALL', 'LooseTkID', 'GENEtaF', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5Pt25', 'GENEtaBC', debug=False)
text_forprint = 'EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4, p_{T}^{L1} #geq 25GeV'
print dir(hsets[0].h_eff)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1],
     do_write=True, write_name='figs/egammaEndcap_EG_EGq5Pt25_GENEtaBC_hEffvPt')



hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5Pt25', ['GENEtaDE'])
print labels
text_forprint = 'EG endcap, 2.4 < |#eta^{GEN}| #leq 3.0, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1],
     do_write=True, write_name='figs/egammaEndcap_EG_EGq5Pt25_GENEtaDE_hEffvPt')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', ['all', 'EGq5'], 'GENEtaBC', debug=False)
text_forprint = 'EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4, p_{T}^{L1} #geq 25GeV'
print dir(hsets[0].h_eff)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', ['all', 'EGq5'], 'GENEtaBC', debug=False)
text_forprint = 'EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4'
print dir(hsets[0].h_eff)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1])



hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5', ['GENEtaDE'])
print labels
text_forprint = 'EG endcap, 2.4 < |#eta^{GEN}| #leq 3.0'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG', 'TkEleEL'], 'EGq5Pt25', 'GENEtaBC')
labels=['calorimeter only', 'track-matched']
legend_position = (0.6, 0.5, 0.86, 0.6)
text_forprint='EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, min_y=0, max_y=1.1, 
     y_axis_label='efficiency', h_lines=[0.95, 1],
     legend_position=legend_position,
     do_write=True, write_name='figs/egammaEndcap_TkEle_EGq5Pt25_GENEtaBC_hEffvPt')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EGBRL'], ['Pt20', 'Pt20LooseTkID'], ['GENEtaF'])
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EGBRL'], ['Pt25', 'Pt25LooseTkID'], ['GENEtaF', 'all'])
for hset in hsets:
    hset.computeEff(rebin=2)

labels=['w/o EG ID', 'w EG ID']
legend_position = (0.6, 0.5, 0.86, 0.6)
text_forprint='EG barrel, |#eta^{GEN}| #leq 1.47, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0.8, max_y=1.02, min_x=0.0, max_x=100, 
     y_axis_label='efficiency', h_lines=[0.95, 1],
     legend_position=legend_position,
     do_write=True, write_name='figs/EgammaBarrelOnlyCaloEff')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EGALL', 'TkEleELALL'], 'Pt25LooseTkID', 'GENEtaF')
labels=['calorimeter only', 'track-matched']
legend_position = (0.6, 0.5, 0.86, 0.6)
text_forprint='EG barrel, |#eta^{GEN}| #leq 1.479, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, min_y=0, max_y=1.1, 
     y_axis_label='efficiency', h_lines=[0.95, 1],
     legend_position=legend_position,
     do_write=True, write_name='figs/egammaBarrel_TkEle_LooseTkIDPt25_GENEtaF_hEffvPt')


## Efficiency vs fBrem

In [ ]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk #stubs > 3")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk EMU")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>2GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq3M1P10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11'], ['PU200'], ['tkEleEE', 'PFtkEleEE'], 'EGq4', 'GENEtaBC', debug=False)
draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-V11'], ['PU200'], ['tkEleEB', 'PFtkEleEB'], 'LooseTkID', 'GENEtaF', debug=False)
draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


## Turn-on curves

In [65]:
['dele-V11'], ['PU200'], ['PFtkEmEE','tkEmEE'], 'EGq5', 'GENEtaBC'

(['dele-V11'], ['PU200'], ['PFtkEmEE', 'tkEmEE'], 'EGq5', 'GENEtaBC')

In [66]:
colors.append(12)
colors.append(13)


In [67]:
rleg_config.marker_styles.append(43)
rleg_config.marker_styles.append(53)
rleg_config.marker_styles.append(63)
rleg_config.marker_styles.append(73)
rleg_config.colors.extend(colors)

AttributeError: 'ColorPalette' object has no attribute 'extend'

In [68]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele-ref', 'ele'], ['PU200'], ['PFNFEgEE', 'PFEgEE', 'EgEE'], ['EGq5Pt10'], 'GENEtaBC', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>10GeV - new emu', 'p_{T}>10GeV - new sim', 'p_{T}>10GeV - TDR sim'])

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFNFEgEE', 'PFEgEE', 'EgEE'], ['EGq5Pt20'], 'GENEtaBC', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>20GeV - new emu', 'p_{T}>20GeV - new sim', 'p_{T}>20GeV - TDR sim'])


hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFNFEgEE','PFEgEE', 'EgEE'], ['EGq5Pt30'], 'GENEtaBC', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>30GeV - new emu', 'p_{T}>30GeV - new sim', 'p_{T}>30GeV - TDR sim'])

# dm.addRatioHisto(0,1)

dm.draw(text=text, y_min=0, y_max=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


No match found for: sample: dele-V11 pu: ['PU200'], tp: ['PFNFEgEE', 'PFEgEE', 'EgEE'], tp_sel: ['EGq5Pt10'], gen_sel: GENEtaBC, classtype: <class 'python.l1THistos.HistoSetEff'>


TypeError: 'NoneType' object is not iterable

In [83]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt10', 'IDTightPPt10'], 'GEN', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt20', 'IDTightPPt20'], 'GEN', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)


hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt30', 'IDTightPPt30'], 'GEN', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)


hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt40', 'IDTightPPt40'], 'GEN', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)


# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEE', 'tkEleEE'], ['EGq5Pt20'], 'GENEtaBC', debug=False)
# dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>20GeV - new sim', 'p_{T}>20GeV - TDR sim'])


# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEE', 'tkEleEE'], ['EGq5Pt30'], 'GENEtaBC', debug=False)
# dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>30GeV - new sim', 'p_{T}>30GeV - TDR sim'])

# dm.addRatioHisto(0,1)

dm.draw(text=text, y_min=0, y_max=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [94]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt10', 'IDTightPPt10'], 'GENEtaBC', debug=False)
dm.addHistos([his.h_eff.h_pt.CreateGraph() for his in hsets], labels)
dm.draw(text=text, y_min=0, y_max=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [92]:

dm = DrawMachine(rleg_config)

# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt10', 'IDTightPPt10'], 'GENEtaBC', debug=False)
# dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt20', 'IDTightPPt20'], 'GENEtaBC', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)


# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt30', 'IDTightPPt30'], 'GENEtaBC', debug=False)
# dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)


# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele'], ['PU200'], ['TkEmL2'], ['Pt40', 'IDTightPPt40'], 'GENEtaBC', debug=False)
# dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], labels)


# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEE', 'tkEleEE'], ['EGq5Pt20'], 'GENEtaBC', debug=False)
# dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>20GeV - new sim', 'p_{T}>20GeV - TDR sim'])


# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEE', 'tkEleEE'], ['EGq5Pt30'], 'GENEtaBC', debug=False)
# dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>30GeV - new sim', 'p_{T}>30GeV - TDR sim'])

# dm.addRatioHisto(0,1)

dm.draw(text=text, y_min=0, y_max=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEB', 'tkEleEB'], ['LooseTkIDPt10'], 'GENEtaF', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>10GeV - new sim', 'p_{T}>10GeV - TDR sim'])

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEB', 'tkEleEB'], ['LooseTkIDPt20'], 'GENEtaF', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>20GeV - new sim', 'p_{T}>20GeV - TDR sim'])


hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEB', 'tkEleEB'], ['LooseTkIDPt30'], 'GENEtaF', debug=False)
dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['p_{T}>30GeV - new sim', 'p_{T}>30GeV - TDR sim'])

# dm.addRatioHisto(0,1)

dm.draw(text=text, y_min=0, y_max=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFEgEE', 'EgEE'], ['EGq5Pt10'], 'GENEtaBC', debug=False)

dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['new sim', 'TDR sim'])

# dm.addRatioHisto(0,1)

dm.draw(text=text, y_min=0, y_max=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'dele-V11', ['PU200'], ['PFtkEleEE', 'tkEleEE'], ['EGq5Pt30'], 'GENEtaBC', debug=False)

dm.addHistos([his.h_ton.h_pt.CreateGraph() for his in hsets], ['new sim', 'TDR sim'])

# dm.addRatioHisto(0,1)

dm.draw(text=text, y_min=0, y_max=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V11', ['PU200'], ['EG'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleEL'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleELALL'], ['Pt15', 'Pt30', 'Pt40'], 'GENEtaF', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


## Scaling

In [ ]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(1, 0, 50)

tf_yc.SetParLimits(3, 0.9, 1)
tf_yc.SetParLimits(0, 0, 1)
tf_yc.SetParLimits(2, 0.1, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [ ]:
sample

In [ ]:
hplot.data[(hplot.data['sample'] == sample) & (hplot.data.tp == 'TkEle')].tp_sel.unique()


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, 'PU0', 'TkEle', 'EGq3Pt15', 'GENEtaBC')


print hsets[0].name

In [ ]:
fake_histoname = uuid.uuid4().hex[:6]


In [ ]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

import array

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


# tps_scaling = ['EG', 'TkEleEL']
# tp_sel_scaling = [('EGq3Pt{}', 'EGq3'), ('EGq4Pt{}', 'EGq4'), ('EGq5Pt{}', 'EGq5')]
# pt_points = [10, 15, 20, 25, 30, 40]

sample = 'ele-V9'
samples_scaling = ['ele-V9', 'dele-NM']
tps_scaling = ['EG', 'EGBRL', 'TkEleEL', 'TkEleELBRL', 'tkEleEE', 'tkEleEB']
tp_sel_scaling = [('EGq5Pt{}', 'EGq5'), 
                  ('Pt{}', 'all'),
                  ('Pt{}LooseTkID', 'LooseTkID'),
                  ('Pt{}Iso0p1', 'Iso0p1')]
pt_points = [10, 15, 20, 25, 30, 40]

# sample = 'calib'
# tps_scaling = ['HMvDRshapeDrCalib']
# tp_sel_scaling = [('EmPt{}', 'Em')]
# pt_points = [10, 20, 30]

for sample in samples_scaling:
    for pu in ['PU0', 'PU200']:
        for gen_sel in ['GENEtaBC', 'GENEtaF']:
            for tp in tps_scaling:
                for tp_sel_strings in tp_sel_scaling:
                    pt_95 = []
                    pt_95_err = []
                    print '--- sample: {} {}, tp: {}, sel: {}, gen_sel: {}'.format(sample, pu, tp, tp_sel_strings[1], gen_sel)
                    for pt in pt_points:
    #                     for (i=0; i<ngr; i++)
    #       grint->SetPoint(i, gr->GetX()[i], 0);

                        tp_sel = tp_sel_strings[0].format(pt)
                        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, gen_sel)
                            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            #                 draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                        if hsets is None:

                            continue
                        print '   pt: {}'.format(pt)
                        #                     if pt == 20:
                        tf_yc.SetParameters(0.6, pt, 0.1, 1.0, 0.0)
    #                     else:
    #                         tf_yc.SetParameters(1, 20, 4, 1.0, 0.0)
                        h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]

                        result = h_eff_vs_pt[0].Fit(tf_yc, 'EMQRS+')

                        grint = ROOT.TGraphErrors(h_eff_vs_pt[0].GetN())
                        stuff.append(grint)
                        grint.SetTitle("Fitted line with .95 conf. band")
                        for point in range(0, h_eff_vs_pt[0].GetN()):
                            grint.SetPoint(point, h_eff_vs_pt[0].GetX()[point], 0)
                        ROOT.TVirtualFitter.GetFitter().GetConfidenceIntervals(grint)


                        print '------- {}'.format( text)
                        print result.Print()
                        pt095 = tf_yc.GetX(0.95)
                        pt_95.append(pt095)

                        prev_err = 0
                        error = 0
                        for idx,point in enumerate(grint.GetX()):
    #                         print idx, point, grint.GetEY()[idx]
                            if pt095 > point:
                                prev_err = grint.GetEY()[idx]
                            else:
                                error = (prev_err+grint.GetEY()[idx])/2
                                break
                        pt_95_err.append(error)
                        print '       pt th: {}, 95% eff: {}, err: {}'.format(pt, pt095, error)
                        #                 points.append((tp, pu, pt, pt095))
                        draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                        ROOT.gStyle.SetOptFit(11111)
                        grint.Draw('same')

                    print len(pt_points)
                    print len(pt_95)
                    print len(pt_95_err)
                    print pt_95_err
                    
                    # this allows to simply skip missing combinations of TP and TPsel 
                    # (see previous continue statement in this cell)
                    if len(pt_points) != len(pt_95):
                        continue
                    graph = ROOT.TGraphErrors(len(pt_points), array.array('d', pt_points), 
                                                              array.array('d', pt_95),
                                                              array.array('d', [0.]*len(pt_points)),
                                                              array.array('d', pt_95_err) )
                    graph.SetMarkerStyle(7)
                    graph.SetMarkerColor(2)
                    stuff.append(graph)

                    hplot.data = hplot.data.append({'sample': sample,
                                                    'pu': pu,
                                                    'tp': tp,
                                                    'tp_sel': tp_sel_strings[1],
                                                    'gen_sel': gen_sel,
                                                    'classtype': fake_histoname,
                                                    'histo': HWrapper(graph),},
                                                    ignore_index=True)

                
                

In [ ]:
hplot.data[hplot.data.classtype == fake_histoname]


In [ ]:
hplot.data[hplot.data.classtype == fake_histoname]['sample'].unique()

In [ ]:
scaling_params = pd.DataFrame(columns=['tp', 'tp_sel', 'params', 'sample'])
ROOT.gStyle.SetOptFit(11111)

for sample in hplot.data[hplot.data.classtype == fake_histoname]['sample'].unique():
    print sample
    for pu in hplot.data[
        (hplot.data.classtype == fake_histoname) & 
        (hplot.data['sample'] == sample)].pu.unique():
        for tp in hplot.data[
            (hplot.data.classtype == fake_histoname) & 
            (hplot.data['sample'] == sample) & 
            (hplot.data.pu == pu)].tp.unique():
            for tp_sel in hplot.data[
                (hplot.data.classtype == fake_histoname) & 
                (hplot.data['sample'] == sample) & 
                (hplot.data.pu == pu) & 
                (hplot.data.tp == tp)].tp_sel.unique():
                for gen_sel in hplot.data[
                (hplot.data.classtype == fake_histoname) & 
                (hplot.data['sample'] == sample) & 
                (hplot.data.pu == pu) & 
                (hplot.data.tp == tp) &
                (hplot.data.tp_sel == tp_sel)].gen_sel.unique():
                    hsets, labels, text = hplot.get_histo(fake_histoname, sample, pu, tp, tp_sel, gen_sel)
                    result = hsets[0].Fit('pol1', 'MES+', '', 10, 40)
                    result.GetParams()
                    scaling_params = scaling_params.append({'sample': sample,
                                                            'pu': pu,
                                                            'tp': tp,
                                                            'tp_sel': tp_sel+gen_sel.split('GEN')[1],
                                                            'params': (result.GetParams()[0], result.GetParams()[1])},
                                                              ignore_index=True)
                    draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')

###### 

In [ ]:
scaling_params

In [ ]:
scaling_params.to_csv(r'scaling_params_{}.csv'.format(version_NM))

In [ ]:
eta_boundaries_fiducial = [-5, -4, -3,-2.5 ,-1.5, -0.5, 0.5, 1.5, 2.5, 3, 4, 5]
eta_boundaries = []
phi_boundaries_fiducial = []


eta_overlap = 0.25
phi_overlap = 0.25
phiSlices = 9

phiWidth = 2*math.pi/phiSlices


phi_boundaries = []


for ieta,eta_low_fiducial in enumerate(eta_boundaries_fiducial):
    if ieta>=len(eta_boundaries_fiducial)-1:
        break
    eta_high_fiducial = eta_boundaries_fiducial[ieta+1]
    print 'fiducial boundaries: {}, {}'.format(eta_low_fiducial, eta_high_fiducial)
    eta_low = eta_low_fiducial - eta_overlap
    eta_high = eta_high_fiducial + eta_overlap
    eta_boundaries.append((eta_low, eta_high))
    
    
    
for iphi in range(0,9):
    phiCenter = (iphi+0.5)*phiWidth-math.pi
    print iphi,phiCenter
    phi_low_fiducial = phiCenter - phiWidth/2.
    phi_high_fiducial = phiCenter+phiWidth/2.
    
    print 'fiducial boundaries: {}, {}'.format(phi_low_fiducial, phi_high_fiducial)
    phi_boundaries_fiducial.append(phi_low_fiducial)
    
    phi_low = phi_low_fiducial - phi_overlap
    if phi_low < -1*math.pi:
        phi_low = math.pi-phi_overlap
    phi_high = phi_high_fiducial + phi_overlap
    if phi_high > math.pi:
        phi_high = -1*math.pi+phi_overlap
    
    phi_boundaries.append((phi_low, phi_high))
    print 'boundaries: {} {}'.format(phi_low, phi_high)
    
    
phi_boundaries_fiducial.append(math.pi)
print "FIDUCIAL"
print eta_boundaries_fiducial
print phi_boundaries_fiducial

print "ACTUAL"
print phi_boundaries
print eta_boundaries

In [ ]:
class pippo:
    def __init__(self, mystring=''):
        print mystring
        return
    
pippo_l = lambda : pippo(mystring="ciccio")

pippo_l()